In [7]:
import tushare as ts
import plotly.express as px
import plotly.graph_objects as pg
pro = ts.pro_api()

In [28]:
code_dict = {'603589':'口子窖',
            '000596':'古井贡酒',
            '603189':'迎驾贡酒',
            '600199':'金种子酒'}

target_code = '603589.sh'
# target_code = '600519.sh'

In [29]:
for k,v in list(code_dict.items()):
    if "." in k: continue
    surfix = '.SH' if k.startswith('6') else '.SZ'
    code_dict.pop(k)
    code_dict[k+surfix] = v
    
ts_codes = ','.join(code_dict.keys())

# 资产负债表

In [30]:
balancesheet = pro.balancesheet(ts_code=target_code)
balancesheet['sumv'] = balancesheet.fillna(0).sum(axis=1)
balancesheet = balancesheet.groupby(['ts_code','end_date']).apply(lambda t:t[t.sumv==t.sumv.max()]).reset_index(drop=True).groupby(['ts_code','end_date']).head(1)
balancesheet.index = balancesheet['end_date']

In [31]:
## 取年报
balancesheet = balancesheet[balancesheet.end_date.apply(lambda x:x.endswith('1231') and x>'2010')]

## 资产负债表中的几个比率

In [48]:
px.line(balancesheet, y='total_assets')

## 货币资金的比率

In [32]:
balancesheet['money_cap_ratio'] = balancesheet['money_cap']/balancesheet['total_assets']
px.line(balancesheet,y='money_cap_ratio')

## 经营资产比率

In [33]:
cols_op = {"notes_receiv":"应收票据",
"accounts_receiv":"应收账款",
"oth_receiv":"其他应收款",
"prepayment":"预付款项",
"div_receiv":"应收股利",
"int_receiv":"应收利息",
"inventories":"存货",
"produc_bio_assets":"生产性生物资产"}

In [34]:
balancesheet['op_ratio'] = balancesheet[cols_op.keys()].sum(axis=1)/balancesheet['total_assets']
px.line(balancesheet,y='op_ratio')

## 生产资产

In [35]:
cols_prod = {"fix_assets":"固定资产",
"cip":"在建工程",
"const_materials":"工程物资",
"intan_assets":"无形资产",
"goodwill":"商誉",
"lt_amor_exp":"长期待摊费用",
"defer_tax_assets":"递延所得税资产"}
# ？递延所得税资产 or 负债？

In [36]:
balancesheet['prod_ratio'] = balancesheet[cols_prod.keys()].sum(axis=1)/balancesheet['total_assets']
px.line(balancesheet,y='prod_ratio')

## 非主营相关资产

In [49]:
cols_oth_ass = {
"trad_asset":"交易性金融资产",
"pur_resale_fa":"买入返售金融资产",
"oth_cur_assets":"其他流动资产",
"fa_avail_for_sale":"可供出售金融资产",
"htm_invest":"持有至到期投资",
"lt_eqt_invest":"长期股权投资",
"invest_real_estate":"投资性房地产"}

In [50]:
balancesheet['oth_ass_ratio'] = balancesheet[cols_oth_ass.keys()].sum(axis=1)/balancesheet['total_assets']
px.line(balancesheet,y='oth_ass_ratio')

In [ ]:
除商誉外的资产负债率
安全性

# 利润表

In [39]:
def process_unique(df):
    df['sumv'] = df.fillna(0).sum(axis=1)
    df = df.groupby(['ts_code','end_date']).apply(lambda t:t[t.sumv==t.sumv.max()]).reset_index(drop=True).groupby(['ts_code','end_date']).head(1)
    df.index = df['end_date']
    return df

In [37]:
income = pro.income(ts_code=target_code)

In [40]:
income = process_unique(income)

In [42]:
income = income[income.end_date.apply(lambda x:x.endswith('1231') and x>'2010')]

In [46]:
px.line(income, y=['total_revenue','operate_profit','n_income'])

In [43]:
income

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,basic_eps,diluted_eps,total_revenue,revenue,...,oth_compr_income,t_compr_income,compr_inc_attr_p,compr_inc_attr_m_s,ebit,ebitda,insurance_exp,undist_profit,distable_profit,sumv
end_date,,,,,,,,,,,,,,,,,,,,,
20101231,603589.SH,20130806,20130806,20101231,1,1,NaN,NaN,1.722510e+09,1.722510e+09,...,None,NaN,NaN,None,NaN,NaN,None,None,None,3.445020e+09
20111231,603589.SH,20140514,20140514,20111231,1,1,0.73,NaN,2.092421e+09,2.092421e+09,...,None,3.966851e+08,3.966851e+08,None,5.443135e+08,5.801483e+08,None,None,None,1.121878e+10
20121231,603589.SH,20150610,20150610,20121231,1,1,0.84,NaN,2.506547e+09,2.506547e+09,...,None,4.559733e+08,4.559733e+08,None,6.303554e+08,6.831043e+08,None,None,None,1.334078e+10
20131231,603589.SH,20150610,20150610,20131231,1,1,0.74,NaN,2.447151e+09,2.447151e+09,...,None,3.981670e+08,3.981670e+08,None,5.794814e+08,6.399910e+08,None,None,None,1.276814e+10
20141231,603589.SH,20150610,20150610,20141231,1,1,0.78,NaN,2.258554e+09,2.258554e+09,...,None,4.221396e+08,4.221396e+08,None,5.715441e+08,6.363179e+08,None,None,None,1.209305e+10
20151231,603589.SH,20160428,20160428,20151231,1,1,1.06,NaN,2.584012e+09,2.584012e+09,...,None,6.053969e+08,6.053969e+08,None,8.256113e+08,9.043166e+08,None,None,None,1.471239e+10
20161231,603589.SH,20170428,20170428,20161231,1,1,1.31,NaN,2.830179e+09,2.830179e+09,...,None,7.834642e+08,7.834642e+08,None,1.028036e+09,1.120075e+09,None,None,None,1.695859e+10
20171231,603589.SH,20180427,20180427,20171231,1,1,1.86,NaN,3.602647e+09,3.602647e+09,...,None,1.113663e+09,1.113663e+09,None,1.502573e+09,1.608298e+09,None,None,None,2.257678e+10
20181231,603589.SH,20190418,20190418,20181231,1,1,2.55,NaN,4.268965e+09,4.268965e+09,...,None,1.532664e+09,1.532664e+09,None,1.967638e+09,2.080060e+09,None,None,None,2.804094e+10
